In [1]:
import gensim
import os
import collections
import smart_open
import random

In [2]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'
print (lee_test_file)
print (lee_train_file)

/home/kamal/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/lee.cor
/home/kamal/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/lee_background.cor


In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [4]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [5]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=10)

/home/kamal/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/kamal/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [6]:
model.build_vocab(train_corpus)

In [18]:
model.vocabulary

In [7]:
%time model.train(train_corpus,total_examples=model.corpus_count,epochs=model.iter)


/home/kamal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


CPU times: user 1.19 s, sys: 16.9 ms, total: 1.21 s
Wall time: 607 ms


In [8]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])

array([-5.1218051e-02,  8.9167543e-02,  5.8330530e-03,  2.1841051e-02,
       -4.9148254e-02,  1.8440602e-02, -1.3963051e-02,  2.4803180e-02,
       -7.3163304e-05, -1.5204197e-02,  5.8169561e-03,  1.3922131e-02,
        6.1183609e-02,  3.0000899e-02,  4.2693544e-02,  2.1754859e-02,
       -6.7927372e-03,  1.3767157e-02,  3.1363377e-03, -5.1696682e-03,
        2.2207364e-03, -1.9045766e-02, -4.9056370e-02,  1.8180920e-02,
       -2.0515705e-02, -6.7289369e-03, -5.3252440e-02, -1.2659206e-02,
        5.1609064e-03, -3.9285216e-02, -1.5433058e-02,  2.2050139e-02,
        3.2933410e-02,  5.2565511e-02, -3.9323494e-02, -9.6941357e-03,
       -1.2293762e-02,  2.2062212e-02,  6.5584704e-02, -4.3579042e-02,
       -2.4192562e-02,  2.8064810e-02,  7.1432577e-03, -3.7213307e-02,
        6.0336523e-02,  1.9008473e-02, -9.4067473e-03,  3.1890817e-02,
        1.2628396e-02,  2.3744132e-02], dtype=float32)

In [9]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [10]:
collections.Counter(ranks)  #96% accuracy

Counter({0: 31,
         1: 33,
         2: 14,
         3: 11,
         4: 8,
         5: 11,
         6: 13,
         7: 6,
         8: 6,
         9: 7,
         10: 11,
         11: 5,
         12: 12,
         13: 5,
         14: 10,
         15: 4,
         16: 4,
         17: 8,
         18: 3,
         19: 4,
         20: 5,
         21: 5,
         22: 3,
         23: 3,
         24: 4,
         25: 1,
         26: 4,
         28: 3,
         29: 1,
         30: 2,
         31: 2,
         32: 2,
         33: 2,
         38: 1,
         39: 4,
         40: 1,
         41: 3,
         42: 1,
         43: 2,
         45: 2,
         47: 3,
         50: 1,
         51: 1,
         54: 1,
         55: 1,
         57: 3,
         58: 3,
         59: 1,
         60: 1,
         63: 1,
         65: 1,
         66: 1,
         68: 2,
         69: 1,
         74: 1,
         76: 1,
         84: 1,
         86: 1,
         88: 1,
         89: 1,
         98: 1,
         101: 1,
        

In [11]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))



Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [12]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (268): «opposition forces claimed to have captured half of kandahar airport after fierce fighting with taliban troops as residents reported further cranking up of us bombardments on the city we have now taken half of the airport said gul lali key lieutenant to former kandahar governor gul agha lali said that their forces had killed foreign taliban fighters in the operation and overrun building that appeared to have been used as an office by members of suspected terrorist mastermind osama bin laden al qaeda network these were of bin laden men from egypt libya and saudi arabia nineteen more were injured lali said from the battleground opposition commanders had earlier expressed confidence that the airport would fall by the end of the day with the taliban defence tactics hamstrung by aerial attacks from us warplanes we can hope but am not per cent sure we will capture it by the end of the evening spokesman for gul agha said after speaking to the opposition leader at noon lo

In [13]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus))
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (29): «man accused of making hidden camera footage up the skirts of women also made child pornography of the worst kind featuring the rape of children as young as police said friday the latest allegations suggest there nothing humorous about voyeurs who some may perceive to be making secret videos as joke staff insp gary ellis said approximately per cent of voyeurs have also committed sexual assault or rape ellis said reading from recently released federal government report on criminal voyeurism»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (206, 0.99898362159729): «the royal commission into the building industry will hold its first public hearings in melbourne today the howard government established the commission in july prior to calling the federal election prompting union claims of political witch hunt but royal commissioner terence cole qc has stressed the independence of his inquiry he will examine claims of corruption coercion and